# Clip and convert rasters to COGs

The purpose of this notebook is to process the rasters used on the ECCC platform by:
1. **Clipping Canada-wide rasters to HBL boundaries** (for datasets that extend beyond the study area)
2. **Converting rasters to COGs** for efficient web access and analysis

The datasets processed are:
- Peat depth: manually downloaded from [Borealis](https://borealisdata.ca/dataset.xhtml?persistentId=doi:10.5683/SP3/TIRAXJ)
- Carbon storage: manually downloaded from [Borealis](https://borealisdata.ca/dataset.xhtml?persistentId=doi:10.5683/SP3/TIRAXJ)
- Annual sub-pixel Landsat surface water maps of the Hudson Bay Lowlands from 1985-2021: downloaded in 01_download_data.ipynb
- Treed Area in Canada (1984-2022): downloaded in 01_download_data.ipynb
- Flood Susceptibility Index (FSI): downloaded in 01_download_data.ipynb
- Hudson Bay Lowland Ecosystem Classification (HBL‑EC): provided by email by the author.

### Set up

In [7]:
import sys
from pathlib import Path

sys.path.append("../src/")
from data_processing.utils import batch_clip_rasters, convert_to_cog

### 1: Clip Canada-wide Rasters to HBL Boundary

Clip the Canada-wide datasets (those starting with "CAN_") to the Hudson Bay Lowlands study area boundary.

In [ ]:
# Define paths
raw_rasters_dir = Path("../data/raw/rasters")
hbl_boundary = Path("../data/processed/vectors/HBL_footprint.geojson")
clipped_dir = Path("../data/processed/rasters/clipped")

# Create output directory
clipped_dir.mkdir(parents=True, exist_ok=True)

# Check if boundary file exists
if hbl_boundary.exists():
    print(f"Found HBL boundary: {hbl_boundary}")
else:
    print(f"HBL boundary not found: {hbl_boundary}")

# Batch clip all Canada-wide rasters (CAN_* pattern)
if hbl_boundary.exists():
    batch_clip_rasters(
        raster_dir=raw_rasters_dir,
        vector_path=hbl_boundary,
        output_dir=clipped_dir
    )

### 2: Convert Rasters to COGs

Convert all rasters (both clipped and HBL-native) to Cloud-Optimized GeoTIFF format for efficient web access.

#### 2.1 Clipped Rasters

In [ ]:
input_dir = clipped_dir
output_dir = Path("../data/processed/rasters/cogs")
output_dir.mkdir(parents=True, exist_ok=True)

clipped_rasters = list(input_dir.glob("*.tif"))

if clipped_rasters:
    print(f"Converting {len(clipped_rasters)} clipped rasters to COGs:")
    for raster_path in clipped_rasters:
        cog_output = output_dir / f"{raster_path.stem}_cog.tif"
        print(f"  {raster_path.name} → {cog_output.name}")
        convert_to_cog(
            geotiff_path=raster_path,
            cog_output_path=cog_output
        )
else:
    print("No clipped rasters found.")

#### 2.2 HBL-Native Rasters

In [ ]:
# Peat & Carbon
raw_dir = Path("../data/raw/rasters")
cog_output_dir = Path("../data/processed/rasters/cogs")

cog_output_dir.mkdir(parents=True, exist_ok=True)

peat_carbon_files = [f for f in raw_dir.glob("*.tif")
                     if "McMaster" in f.name]

for tif_path in peat_carbon_files:
    cog_output = cog_output_dir / f"{tif_path.stem}_cog.tif"

    print(f"Converting {tif_path.name} → {cog_output.name}")

    convert_to_cog(
        geotiff_path=tif_path,
        cog_output_path=cog_output,
    )

In [ ]:
# Surface Water
raw_dir = Path("../data/raw/rasters/HBL_SurfaceWater_1985_2021")
cog_output_dir = Path("../data/processed/rasters/cogs/HBL_SurfaceWater_1985_2021")

cog_output_dir.mkdir(parents=True, exist_ok=True)

for tif_path in raw_dir.glob("*.tif"):
    cog_output = cog_output_dir / f"{tif_path.stem}_cog.tif"

    print(f"Converting {tif_path.name} → {cog_output.name}")

    convert_to_cog(
        geotiff_path=tif_path,
        cog_output_path=cog_output,
    )

In [ ]:
# Ecosystem Classification
raw_dir = Path("../data/raw/rasters/HBL_V1_DataPackage_01152026")
cog_output_dir = Path("../data/processed/rasters/cogs")

cog_output_dir.mkdir(parents=True, exist_ok=True)

for tif_path in raw_dir.glob("*.tif"):
    cog_output = cog_output_dir / f"{tif_path.stem}_cog.tif"

    print(f"Converting {tif_path.name} → {cog_output.name}")

    convert_to_cog(
        geotiff_path=tif_path,
        cog_output_path=cog_output,
    )